In [1]:
import numpy as np

In [2]:
def show(x,name) :
    print(x); print('%s shape:'%name, end=' '); print(x.shape)


### Data 선언

In [19]:
N = 2       # input
n_ic = 2    # input(filter) channel
n_oc = 3    # output channel
n_i = 4     # input(row,col) size
n_f = 3     # filter(row,col) size
p = 0       # padding size
s = 1       # stride size
n_o = int((n_i + 2*p - n_f)/s) + 1  # output size
# X(N, n_ic, n_i, n_i)
X11 = np.array([[1,2,3,0],[0,1,2,3],[3,0,1,2],[2,3,0,1]])
X12 = np.array([[5,6,8,0],[0,6,2,3],[5,2,1,9],[4,3,1,1]])
X1 = np.array([X11,X12])
X21 = np.array([[1,2,1,0],[0,2,2,3],[3,0,9,2],[1,3,0,2]])
X22 = np.array([[5,6,8,0],[2,5,2,3],[2,2,6,9],[4,2,1,4]])
X2 = np.array([X21,X22])
X = np.array([X1,X2])
X = np.pad(X,((0,0),(0,0),(p,p),(p,p)),constant_values=(0,))
show(X,'X')
# W(n_oc, n_ic, n_f, n_f)
W11 = np.array([[2,0,1],[0,1,2],[1,0,2]])
W12 = np.array([[1,2,1],[0,2,0],[1,0,1]])
W1 = np.array([W11,W12])
W21 = np.array([[1,1,1],[1,0,1],[1,2,1]])
W22 = np.array([[1,1,1],[0,1,1],[0,2,2]])
W2 = np.array([W21,W22])
W31 = np.array([[2,1,1],[1,0,0],[1,0,1]])
W32 = np.array([[2,0,1],[2,2,0],[0,2,0]])
W3 = np.array([W31,W32])
W = np.array([W1,W2,W3])
show(W,'W')
# output(N, n_oc, n_o, n_o)


[[[[1 2 3 0]
   [0 1 2 3]
   [3 0 1 2]
   [2 3 0 1]]

  [[5 6 8 0]
   [0 6 2 3]
   [5 2 1 9]
   [4 3 1 1]]]


 [[[1 2 1 0]
   [0 2 2 3]
   [3 0 9 2]
   [1 3 0 2]]

  [[5 6 8 0]
   [2 5 2 3]
   [2 2 6 9]
   [4 2 1 4]]]]
X shape: (2, 2, 4, 4)
[[[[2 0 1]
   [0 1 2]
   [1 0 2]]

  [[1 2 1]
   [0 2 0]
   [1 0 1]]]


 [[[1 1 1]
   [1 0 1]
   [1 2 1]]

  [[1 1 1]
   [0 1 1]
   [0 2 2]]]


 [[[2 1 1]
   [1 0 0]
   [1 0 1]]

  [[2 0 1]
   [2 2 0]
   [0 2 0]]]]
W shape: (3, 2, 3, 3)


### Homework1

In [4]:
Y_ = np.zeros([N,n_oc,n_ic,n_o,n_o])
for i in range(n_o) :
    i1 = s*i; i2 = i1 + n_f
    for j in range(n_o) :
        j1 = s*j; j2 = j1 + n_f
        for k1 in range(n_oc) :
            Y_[:,k1,:,i,j] = np.sum( X[:,:,i1:i2,j1:j2]*W[k1], axis=(2,3) )
Y = np.sum(Y_,axis=2)
show(Y,'Y')

[[[[58. 53.]
   [29. 34.]]

  [[45. 52.]
   [34. 37.]]

  [[45. 40.]
   [30. 34.]]]


 [[[73. 53.]
   [44. 57.]]

  [[60. 77.]
   [46. 49.]]

  [[53. 47.]
   [26. 45.]]]]
Y shape: (2, 3, 2, 2)


### Homework2

In [20]:
W_col = W.reshape([n_oc,n_ic*n_f*n_f]); W_col = W_col.T
X_col = np.empty([N*n_o*n_o,n_ic*n_f*n_f])
for i in range(n_o) :
    i1 = s*i; i2 = i1 + n_f
    for j in range(n_o) :
        j1 = s*j; j2 = j1 + n_f
        for k in range(N) :
            idx = N*n_o*k + n_o*i + j
            X_col[idx,:] = X[k,:,i1:i2,j1:j2].flatten()
Y = X_col@W_col
Y = Y.reshape([N,-1,n_oc])
Y = np.transpose(Y,[0,2,1])
Y = Y.reshape([N,n_oc,n_o,n_o])
show(Y,'Y')

[[[[58. 53.]
   [29. 34.]]

  [[45. 52.]
   [34. 37.]]

  [[45. 40.]
   [30. 34.]]]


 [[[73. 53.]
   [44. 57.]]

  [[60. 77.]
   [46. 49.]]

  [[53. 47.]
   [26. 45.]]]]
Y shape: (2, 3, 2, 2)


In [18]:
W_col = W.reshape([n_oc, n_ic*n_f*n_f])
W_col = W_col.T
X_col = np.empty([N*n_o*n_o, n_ic*n_f*n_f])
for i in range(n_o) :
    i1 = s*i; i2 = i1 + n_f
    for j in range(n_o) :
        j1 = s*j; j2 = j1 + n_f
        idx = n_o*i + j 
        for k in range(N) : 
            X_col[idx+k*n_o*n_o,:] = X[k,:,i1:i2,j1:j2].flatten()
Y = X_col@W_col
Y = np.split(Y,2,axis=0)
Y = np.array(Y).swapaxes(1,2)
Y = Y.reshape(N,n_oc,n_o,n_o)
show(Y,'Y')

[[[[58. 53.]
   [29. 34.]]

  [[45. 52.]
   [34. 37.]]

  [[45. 40.]
   [30. 34.]]]


 [[[73. 53.]
   [44. 57.]]

  [[60. 77.]
   [46. 49.]]

  [[53. 47.]
   [26. 45.]]]]
Y shape: (2, 3, 2, 2)


### Data2

In [6]:
N = 2       # input data
n_ic = 2    # inputdata channel
n_i = 4     # input data size
n_p = 2     # pooling mask size
p = 0       # padding
s = 2       # stride
n_o = int((n_i + 2*p - n_p)/s) + 1

# X[N, n_ic, n_i, n_i]
X11 = np.array([[1,2,3,0],[0,1,2,3],[3,0,1,2],[2,3,0,1]])
X12 = np.array([[5,6,8,0],[0,6,2,3],[5,2,1,9],[4,3,1,1]])
X1 = np.array([X11,X12])
X21 = np.array([[1,2,1,0],[0,2,2,3],[3,0,9,2],[1,3,0,2]])
X22 = np.array([[5,6,8,0],[2,5,2,3],[2,2,6,9],[4,2,1,4]])
X2 = np.array([X21,X22])
X = np.array([X1,X2])
X = np.pad(X,((0,0),(0,0),(p,p),(p,p)),constant_values=(0,))
show(X,'X')

[[[[1 2 3 0]
   [0 1 2 3]
   [3 0 1 2]
   [2 3 0 1]]

  [[5 6 8 0]
   [0 6 2 3]
   [5 2 1 9]
   [4 3 1 1]]]


 [[[1 2 1 0]
   [0 2 2 3]
   [3 0 9 2]
   [1 3 0 2]]

  [[5 6 8 0]
   [2 5 2 3]
   [2 2 6 9]
   [4 2 1 4]]]]
X shape: (2, 2, 4, 4)


### Homework3

In [7]:
Y_ = np.zeros([N, n_ic, n_o, n_o])

for i in range(n_o) :
    i1 = s*i
    i2 = i1 + n_p
    for j in range(n_o) :
        j1 = s*j
        j2 = j1 + n_p
        for n in range(N) :
            for w in range(n_ic) :
                Y_[n,w,i,j] = np.max(X[n,w,i1:i2,j1:j2])
show(Y_,'Y')

[[[[2. 3.]
   [3. 2.]]

  [[6. 8.]
   [5. 9.]]]


 [[[2. 3.]
   [3. 9.]]

  [[6. 8.]
   [4. 9.]]]]
Y shape: (2, 2, 2, 2)


### Homework 4

In [8]:
X_col = np.zeros([N*n_ic*n_p*n_p,n_p*n_p])
X_ = X.reshape(N*n_ic,n_i,n_i)
for i in range(n_p) :
    i1 = s*i
    i2 = i1 + n_p
    for j in range(n_p) :
        j1 = s*j
        j2 = j1 + n_p
        idx = n_p*i + j
        for k in range(N*n_ic) : 

            X_col[idx+4*k,:] = X_[k,i1:i2,j1:j2].flatten()
Y = X_col.max(axis=1)
Y = np.split(Y,2,axis=0)
Y = np.array(Y)
Y = Y.reshape(N,n_ic,n_p,n_p)
show(Y,'Y')

[[[[2. 3.]
   [3. 2.]]

  [[6. 8.]
   [5. 9.]]]


 [[[2. 3.]
   [3. 9.]]

  [[6. 8.]
   [4. 9.]]]]
Y shape: (2, 2, 2, 2)
